In [39]:
import pandas as pd
import numpy as np
import importlib

# graphs
import chart_studio.plotly as py
import plotly.graph_objs as go

import util.Plotly as plots

In [47]:
importlib.reload(plots)

<module 'util.Plotly' from 'D:\\scul\\uaic-ub\\licenta\\code\\repr\\util\\Plotly.py'>

# Data

In [4]:
csv_filename = r'data/final_coordinates.csv'

In [5]:
data_df = pd.read_csv(csv_filename)

In [6]:
## convert Broadman Area column values to string
data_df['Broadman Area'] = data_df['Broadman Area'].astype(str)
## add index column
data_df['index_col'] = data_df.index

In [7]:
data_df.head()

,importance,author,title,table_name,keywords,Left/Right,Name,Broadman Area,t,X(R),Y(A),Z(S),z-score,NV,p value,cluster size,index_col
0,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,R.,amygdala,nan,6.58,18.0,-7.0,-28.0,NaN,NaN,NaN,NaN,0
1,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L.,amygdala,nan,6.47,-17.0,-3.0,-14.0,NaN,NaN,NaN,NaN,1
2,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L.,hippocampus,nan,5.40,-28.0,-24.0,-14.0,NaN,NaN,NaN,NaN,2
3,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,L.,orbitofrontal cortex,11,5.31,-21.0,21.0,-25.0,NaN,NaN,NaN,NaN,3
4,++,Cohen et al,Individual differences in extraversion and dop...,Table 1 Study 2 reward - no reward,fMRI; MNI; extraversion; dopamine; money rewar...,R.,precuneus,23,5.21,4.0,-56.0,35.0,NaN,NaN,NaN,NaN,4


In [8]:
data_df.shape

(656, 17)

In [9]:
data_df.columns.values.tolist()

['importance',
 'author',
 'title',
 'table_name',
 'keywords',
 'Left/Right',
 'Name',
 'Broadman Area',
 't',
 'X(R)',
 'Y(A)',
 'Z(S)',
 'z-score',
 'NV',
 'p value',
 'cluster size',
 'index_col']

In [10]:
coordinates = data_df[['X(R)', 'Y(A)', 'Z(S)']]


# Separate BA of interest

In [14]:
# df_ba = data_df[data_df['Broadman Area'].str.contains('8|9|10|11|12|13|14|24|25|32|33|44|45|46|47')]
# df_ba = data_df[data_df['Broadman Area'].str.contains('4|6|8|9|10|11|12|25')]
df_ba = data_df[data_df['Broadman Area'].str.contains('^4$|^6$|^8$|^9$|^10$|^11$|^12$|^25$', regex=True)]
df_ba_nan = data_df[data_df['Broadman Area'].str.contains('^nan$', regex=True)]
assert (len(df_ba_nan['Broadman Area'].unique()) == 1)

In [41]:
data_dict_coords = plots.create_data(df=df_ba, grouping_col='Broadman Area')
data_dict_nan = plots.create_data(df=df_ba_nan, grouping_col='Broadman Area', check_none=True, color_nan='#ebeded')


In [64]:
df = df_ba
col1 = 'X(R)'
col2 = 'Y(A)'
col3 = 'Z(S)'
Xn = df[col1].to_numpy()  # x-coordinates of nodes
Yn = df[col2].to_numpy()  # y-coordinates
Zn = df[col3].to_numpy()

In [65]:
# color V1
ba = df['Broadman Area'].unique()
ba_idx = {str(x): ba.tolist().index(x) for x in ba}
no_colors = len(ba)
color_group_id = []
labels = []
for i, val in enumerate(df[['Broadman Area', 'index_col', 'Name']].to_numpy()):
    color_group_id.append(ba_idx[str(val[0])])
    labels.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]), name=val[2], ba=str(val[0])))

In [66]:
# color V2
## Alphabet maximum number of colors = 26
ba = df['Broadman Area'].unique()
ba_idx = {str(x): ba.tolist().index(x) for x in ba}
no_colors = len(ba)
color_group = []
labels = []

#colors
import plotly.express as px

color_list = px.colors.qualitative.Alphabet[:no_colors]
for i, val in enumerate(df[['Broadman Area', 'index_col', 'Name']].to_numpy()):
    # print(i,val[0])
    if val[0] == 'nan':
        color_group.append('#ebeded')
    else:
        color_group.append(color_list[ba_idx[str(val[0])]])
    labels.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]), name=val[2], ba=str(val[0])))

In [67]:
data_dict_coords = dict(
    Xn=Xn,
    Yn=Yn,
    Zn=Zn,
    color_group=color_group,
    labels=labels
)



## Nan data

In [71]:
df_ba_nan = data_df[data_df['Broadman Area'].str.contains('^nan$', regex=True)]
assert (len(df_ba_nan['Broadman Area'].unique()) == 1)
col1 = 'X(R)'
col2 = 'Y(A)'
col3 = 'Z(S)'

Xn_nan = df_ba_nan[col1].to_numpy()
Yn_nan = df_ba_nan[col2].to_numpy()
Zn_nan = df_ba_nan[col3].to_numpy()

#labels for nan
labels_nan = []
for i, val in enumerate(df_ba_nan[['Broadman Area', 'index_col', 'Name']].to_numpy()):
    labels_nan.append('index: {index}; ba: {ba}; name: {name}'.format(index=str(val[1]), name=val[2], ba=str(val[0])))
color_nan = '#ebeded'

data_dict_nan = dict(
    Xn=Xn_nan,
    Yn=Yn_nan,
    Zn=Zn_nan,
    color_group=color_nan,
    labels=labels_nan
)


In [45]:
data_dict_coords_all = [data_dict_coords, data_dict_nan]

In [46]:
plots.plot_coordinates_plotly(data=data_dict_coords_all,
                              data_source_name=csv_filename,
                              title='Center Coordinates')




[{'Xn': array([-21.   ,  25.   ,  53.   ,  53.   ,  25.   ,  21.   , -10.   ,
        25.   ,   0.   , -28.283, -34.343, -51.515,  10.101, -11.111,
        40.404,  15.152, -33.333,  -6.061,  -3.03 ,  18.   ,   0.   ,
         9.   ,  45.   ,  39.   ,  51.   ,  40.   ,   4.04 , -24.242,
         0.   , -59.596,  53.535, -36.364,  33.333, -50.505,   3.03 ,
        -6.061,  36.364,  32.   , -24.   ,   6.   ,  12.   ,  -2.   ,
        10.   ,   0.   ,  -2.   , -18.   , -26.   , -24.   ,   0.   ,
        12.   ,   8.   ,  -6.   ,   4.   ,   0.   ,   0.   ,   0.   ]), 'Yn': array([ 21.   ,  25.   , -17.   ,   7.   ,  49.   ,  39.   ,  42.   ,
       -10.   ,  -6.643, -58.794, -17.455,  -4.069,  46.538,  45.509,
        36.448,  63.322,  44.018,  40.617,  62.242,   6.   ,  51.   ,
        57.   ,  36.   ,  16.   ,  16.   ,   6.   ,  56.013,  40.154,
        54.366,   9.06 ,   3.448,  35.727,  39.28 ,   0.514,  60.389,
        59.77 ,  35.626,   4.   ,   6.   ,  40.   ,  58.   ,  68.   ,
    